In [2]:

import tkinter as tk
from tkinter import messagebox, simpledialog, ttk
from ttkthemes import ThemedTk
import cv2
import face_recognition
import numpy as np
import os

# Créer un dossier pour les employés s'il n'existe pas
if not os.path.exists('Employees'):
    os.makedirs('Employees')

# Identifiants de l'administrateur (vous pouvez les remplacer par les vôtres)
ADMIN_LOGIN = "admin"
ADMIN_PASSWORD = "123"

def register_employee(name, image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(image_rgb)
    
    if len(encodings) == 0:
        messagebox.showerror("Erreur", "Aucun visage détecté dans l'image.")
        return
    
    image_encoding = encodings[0]
    np.save(f'Employees/{name}.npy', image_encoding)
    cv2.imwrite(f'Employees/{name}.jpg', image)
    messagebox.showinfo("Succès", f"L'employé {name} a été enregistré.")

def capture_image():
    cap = cv2.VideoCapture(0)
    captured_image = None
    if not cap.isOpened():
        messagebox.showerror("Erreur", "Impossible d'ouvrir la caméra.")
        print("Erreur: Impossible d'ouvrir la caméra.")
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Erreur", "Impossible de lire le flux vidéo de la caméra.")
            print("Erreur: Impossible de lire le flux vidéo de la caméra.")
            break
        cv2.imshow('Capture Image', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            captured_image = frame
            break
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return captured_image

def recognize_employee(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(image_rgb)
    
    if len(encodings) == 0:
        return None
    
    image_encoding = encodings[0]
    
    for file in os.listdir('Employees'):
        if file.endswith('.npy'):
            employee_encoding = np.load(os.path.join('Employees', file))
            results = face_recognition.compare_faces([employee_encoding], image_encoding)
            if results[0]:
                return os.path.splitext(file)[0]  # Nom de l'employé sans extension
    return None

def list_all_employees():
    employee_names = []
    for file in os.listdir('Employees'):
        if file.endswith('.npy'):
            name = os.path.splitext(file)[0]
            employee_names.append(name)
    return employee_names

def delete_employee(name):
    try:
        os.remove(f'Employees/{name}.npy')
        os.remove(f'Employees/{name}.jpg')
        messagebox.showinfo("Succès", f"L'employé {name} a été supprimé.")
    except FileNotFoundError:
        messagebox.showerror("Erreur", "Employé non trouvé.")

def show_employee_photos():
    # Récupérer la liste des noms des employés
    employee_names = list_all_employees()

    if not employee_names:
        messagebox.showinfo("Liste des employés", "Aucun employé enregistré.")
        return

    photos_window = tk.Toplevel(root)
    photos_window.title("Liste des employés")
    photos_window.geometry("600x400")

    for name in employee_names:
        label_name = ttk.Label(photos_window, text=name)
        label_name.pack(pady=5)

def admin_login():
    login = simpledialog.askstring("Connexion Admin", "Login:")
    password = simpledialog.askstring("Connexion Admin", "Mot de passe:", show='*')

    if login == ADMIN_LOGIN and password == ADMIN_PASSWORD:
        admin_panel()
    else:
        messagebox.showerror("Erreur", "Login ou mot de passe incorrect.")

def admin_panel():
    admin_window = tk.Toplevel(root)
    admin_window.title("Espace Administrateur")
    admin_window.geometry("900x600")

    def add_employee():
        name = simpledialog.askstring("Enregistrement", "Entrez le nom de l'employé:")
        if name:
            messagebox.showinfo("Info", "Appuyez sur 's' pour capturer l'image. Appuyez sur 'q' pour quitter.")
            employee_image = capture_image()
            if employee_image is not None:
                register_employee(name, employee_image)
            else:
                messagebox.showwarning("Avertissement", "Capture d'image annulée.")

    def remove_employee():
        name = simpledialog.askstring("Suppression", "Entrez le nom de l'employé à supprimer:")
        if name:
            delete_employee(name)

    def show_employees():
        show_employee_photos()

    btn_add = ttk.Button(admin_window, text="Ajouter un employé", command=add_employee)
    btn_add.pack(pady=10)

    btn_remove = ttk.Button(admin_window, text="Supprimer un employé", command=remove_employee)
    btn_remove.pack(pady=10)

    btn_show = ttk.Button(admin_window, text="Afficher les employés", command=show_employees)
    btn_show.pack(pady=10)

def employee_login():
    messagebox.showinfo("Info", "Appuyez sur 's' pour prendre une photo et vous connecter. Appuyez sur 'q' pour quitter.")
    login_image = capture_image()
    if login_image is not None:
        employee_name = recognize_employee(login_image)
        if employee_name:
            messagebox.showinfo("Succès", f"Bonjour, {employee_name}!")
        else:
            messagebox.showwarning("Erreur", "Employé non reconnu.")
    else:
        messagebox.showwarning("Avertissement", "Capture d'image annulée.")

# Interface principale avec thème
root = ThemedTk(theme="arc")
root.title("Système de reconnaissance des employés")
root.geometry("900x600")

# Style
style = ttk.Style()
style.configure('TButton', font=('Helvetica', 12), padding=10)
style.configure('TLabel', font=('Helvetica', 14))

# Widgets
title_label = ttk.Label(root, text="Système de reconnaissance des employés", anchor="center")
title_label.pack(pady=20)

btn_admin_login = ttk.Button(root, text="Connexion Admin", command=admin_login)
btn_admin_login.pack(pady=10)

btn_employee_login = ttk.Button(root, text="Connexion Employé", command=employee_login)
btn_employee_login.pack(pady=10)

root.mainloop()


Erreur: Impossible de lire le flux vidéo de la caméra.
